# Databaker - Intro, part 1

Michael Adams, 04/05/2017

A simple introduction to the databaker python library using a very basic hand crafted spreadsheet.

This notebook is intended as an introduction to help explain what databaker is and what it can do rather than a complete showcase of capability (if there's interest I'll build more examples for that).

Please note - I'm going to make heavy use of excel images to explain concepts in this walkthrough, but databaker is purely a python library so you will not use excel  at all in practice (or even need it installed).



## What is Databaker?

Databaker is a python library for 'baking' irregularly formatted data (in the form of hand crafted spreadsheets) into strictly dimensioned machine readable datasets.

The image below show some data (coloured in yellow) both in a spreadsheet and after the databaker process.

---

![alt text](images/both.png)

---

Take a moment and compare the two. You can see in the output (lower) picture how the relationship between the observations (the counts, values etc being measured) and their dimensions (which explain WHAT we're measuring) is obvious and clearly defined. This helps make it machine readable.

In the upper picture (the hand crafted spreadsheet) these relationships are abstracted and represented visually, this sort of abstact is of limited use to a computerised data system.

A key thing to understand at this point is that databaker recipes (i.e scripts) are repeat use. If you were to add additional groups to the above spreadsheet the recipe would still run and would accomodate the extra data seemlessly.

In ONS context, if you add another month, quarter or year onto a spreadsheet no code modification would be required to databake it.


# How does it work?

There are four phases to the databaker process:
    
1.) Load a spreadsheet, and select the tabs you're interested in.

2.) Select and group the cells.

3.) Define the dimensions (the relationships between these groups of cells).

4.) Output the result as a CSV.

We'll take a look at each of these steps in turn and run a few code example. We'll use these to build up to the final recipe.



# First - the data

We'll use the example spreadsheet show below. Its from a tab called "Groups" in a spreadsheet named "databakerExamples.xlsx".

![alt text](images/source.png)



# 1.) Load a spreadsheet, and select the tabs you're interested in.

We need to import databaker and define the name of the input and output files we'll be using, then load the tabs we're interested in. 

NOTE - you can name the output file later if you like, but convention-wise its easier to do it at the start of a recipe, so the things most subject to change are in the same place.

Providing you have the 'databakerExample.xlsx' file in the same directory as this file you can select and run the below code - you should get a few lines of output confirming filesize and the names ot tables loaded.

Any line starting # is just a comment rather than code.


In [1]:

# import the databaker framework - i.e tell the computer we're using databaker
from databaker.framework import *

# name the input and output file
inputfile = 'databakerExample.xlsx'
outputfile ='ExampleDatabakerOutput.csv'

# load the tabs we want
tabsWeWant = ['Groups']
tabs = loadxlstabs(inputfile, tabsWeWant)


Loading databakerExample.xlsx which has size 9932 bytes
Table names: ['Groups']


## 2.) Select and group the cells.

Databaker uses dot notation for selecting cells. Basically, you start with a selection of data and each additional .command modifies that selection somehow.

In other words - each line is a sequence of commands executed one after the other.

Here's a complete line:

---

![alt text](images/line1.png)

We're now going to walk through this line of commands. We'll adress each .command in turn, highlighting which "cells" are selected as each command executes.


# 'tab' command

This is the first command and the standard starting point - literally select all cells in the tab we are looking at.

so:
    
![alt text](images/line2.png)

gets us to:
    
![alt text](images/select1.png)

---

## excel_ref('C5')

means .... FROM THE CURRENT SELECTION ..... get us any cell with an 'excel reference' (letter/number) of 'C5'

so:

![alt text](images/line3.png)

gets us to:
    
![alt text](images/select2.png)

---

## expand(DOWN)

means .... FROM THE CURRENT SELECTION ..... expand downwards and select all cells, including current selection.

NOTE - there is also a ".fill" command that does the same thing but does NOT include the current selection.

so:

![alt text](images/line4.png)

gets us to:
    
![alt text](images/select3.png)


## expand(RIGHT)

means .... FROM THE CURRENT SELECTION ..... expand right selecting all cells, including current selection.

NOTE - the "FROM CURRENT SELECTION" is important here. We select the cells to the right of ALL the cells currently selected.

so:

![alt text](images/line5.png)

gets us to:
    
![alt text](images/select4.png)

## is_not_blank()

means .... FROM THE CURRENT SELECTION ..... only keep the cells, if each cell IS NOT BLANK.

so:

![alt text](images/line6.png)

gets us to:
    
![alt text](images/select5.png)



# Selection - Putting it all together

The following images use coloured cells and commands to highlight all four data selections we'll use in this databaker recipe.

Fistly, here is the data we will be selecting:

---

![alt text](images/allColours.png)

---

Now, the commands you would use to select them.

Please note: The words to the left of the = are just variable names I've chosen. You can use whatever you like but something relatively descriptive is always a good idea.


![alt text](images/colCommands.png)

Take a moment and run through the logic of these selections, making sure you understand how each command resulted in the selection of cells with the matching colour.




# 3.) Define the dimensions (the relationships between these groups of cells).


At this stage we have all the data selected into groups of cells. We now need to define the relationships between these groups of cells.

There is one Golden Rule that applies here:


## you ALWAYS define relationships in terms of  each groups relationship to the observations.

 
 
let's look at an example:

Whats the relationship between "assets" and "observations"? - its that FOR ALL OBSERVATIONS the correct asset is directly above.

---

![alt text](images/relations.png)


Using databaker syntax we write this as:



## HDim(assets, "Assets", DIRECTLY, ABOVE)



this command breaks down as follows:

---

![alt text](images/commandStuff.png)

---

The following is how you would define all the relationships in this example:


In [12]:

# Dont try and run the code in the cell please. It will not work as a stand alone.

dimensions = [
          HDim(assets, "Assets", DIRECTLY, ABOVE), 
          HDim(names, "Name", DIRECTLY, LEFT), 
          HDim(group, "Group", CLOSEST, ABOVE) 
             ]


Let's looks at this in context of our coloured images.

The key thing is to remember the mustard/yellow colour are the observations and all dimensions are defined relative to them.
    
---

![alt text](images/allColours.png)

---
Take a moment and make sure you understand the logic behind how we've established the relationships.

In the case of the "CLOSEST, ABOVE" commands used to define the groups dimension it works as follows:

...for each observation, we want the cell in our groups selection that is the closest one above the observation cell in question. IMPORTANT - being on the same line is always counted as being CLOSEST, ABOVE.


---

# 4.) Output the result as a CSV.




The final part of a databaker script is boiler plated - it never changes. 

As such we'll move straight to the final example recipe, you can see all the commands in context and hopefully they'll make a degree of sense now.

Remember, the idea of this document is to give you a good overview on how the underlying logic and process works - nothing more. It's just a starting point.

NOTE - I'll make heavy use of comments in this script (lines beginning #).



In [1]:

# import the databaker framework - i.e tell the computer we're using databaker
from databaker.framework import *

# name the input and output file
inputfile = 'databakerExample.xlsx'
outputfile ='ExampleDatabakerOutput.csv'

# load the tabs we want
tabsWeWant = ['Groups']
tabs = loadxlstabs(inputfile, tabsWeWant)


# define a list. this is mandatory - for later
conversionsegments = []

# for each of the selected tabs....do everything thats indented (in this case we only have 1 tab, but that isn't common so we'll stick with the typical approach)
for tab in tabs:       

    # define a selection of cells as the observations
    observations = tab.excel_ref('C5').expand(DOWN).expand(RIGHT).is_not_blank()
    
    # define other selections of cells to be our dimensions
    assets = tab.excel_ref('C3').expand(RIGHT).is_not_blank()
    names = tab.excel_ref('B5').expand(DOWN).is_not_blank()
    group = tab.excel_ref('A5').expand(DOWN).is_not_blank()

    # define the relationships of the cells selected as dimensions (relative to the observations)
    dimensions = [
              HDim(assets, "Assets", DIRECTLY, ABOVE), 
              HDim(names, "Name", DIRECTLY, LEFT), 
              HDim(group, "Group", CLOSEST, ABOVE) 
                 ]
    
    # Now we process these relationship for this tab (this code never changes)
    conversionsegment = ConversionSegment(tab, dimensions, observations) # < --- processing
    conversionsegments.append(conversionsegment) # <-- adding result of processing this tab to our list
    
# print it all to csv (this code never changes)
writetechnicalCSV(outputfile, conversionsegments)


Loading databakerExample.xlsx which has size 10098 bytes
Table names: ['Groups']
writing 1 conversion segments into C:\Databaker_Walkthrough_1\ExampleDatabakerOutput.csv
conversionwrite segment size 24 table 'Groups'; 



# Input and Output

Here's the data before and after.

---
![alt text](images/source.png)
---
![alt text](images/output.png)
